## COMP8210 Assignment 2

# Task 1: Data Curation
Data was cleaned to make sure it is valid in json format.

In [1]:
# read the file
file = open("10000 tweets 1.json", 'r')
file_contents = file.read()
#save the lines in a list
contents_list = file_contents.splitlines()
file.close()

In [2]:
# remove all the white space
contents_list = [x.strip() for x in contents_list if x.strip()]

In [3]:
# check the content
contents_list[:10]

['/* 10000 Tweets */',
 '/* 1 */',
 '{',
 '"_id" : ObjectId("abe415aa5e7133a4a61c0d8317875"),',
 '"id" : "149715690143449899009",',
 '"objectType" : "activity",',
 '"actor" : {',
 '"objectType" : "person",',
 '"id" : "10243188921458",',
 '"link" : "http://www.twitter.com/losebabyweight1",']

In [4]:
# remove unnecessary words
contents_list.remove("/* 10000 Tweets */")
contents_list.remove("/* 1 */")

In [5]:
# check the content
contents_list[:10]

['{',
 '"_id" : ObjectId("abe415aa5e7133a4a61c0d8317875"),',
 '"id" : "149715690143449899009",',
 '"objectType" : "activity",',
 '"actor" : {',
 '"objectType" : "person",',
 '"id" : "10243188921458",',
 '"link" : "http://www.twitter.com/losebabyweight1",',
 '"displayName" : "losebabyweight",',
 '"postedTime" : "2010-09-09T22:40:10.000Z",']

In [6]:
# replace lines started with "/* " with comma to separate each tweets
for i, item in enumerate(contents_list):
    if "/* " in item:
        contents_list[i] = ","

In [7]:
# remove ObjectId items which are auto-generated ids in MongoDB and needed to be removed to prevent errors
for i, item in enumerate(contents_list):
    if "ObjectId" in item:
        contents_list[i] = ""

In [8]:
# replace items which store data as 'NumberLong(numbers)' with pure integers by removing the characters 
for i, item in enumerate(contents_list):
    if "NumberLong" in item:
        text = item.replace("NumberLong(", "")
        contents_list[i] = text.replace(")", "")

In [9]:
# join all the lines into a string
data = ''
for x in contents_list:
    data += x

In [10]:
# add square brackets to turn it into a list of dictionaries 
data = "[" + data + "]"

In [11]:
# check the type
type(data)

str

In [12]:
# write into file
with open('cleaned_10000_tweets.json', 'w+',encoding='utf-8-sig') as f:
        f.write(data)
# if want to view as a list of dictionaries in Visual Studio Code, 
# use cmd A to select all, then press cmd K then cmd F to auto indent the codes

# Task 2: Graph Data Model
 Cypher codes are used to create the Graph Data Model for Tweets in Twitter in Neo4j.

The final graph data model is as below:
``CALL db.schema.visualization``
![graph data model](https://i.im.ge/2022/09/21/1DpuaL.graph-data-model.png)

### Connect to Neo4j and using Neo4j Desktop

In [23]:
# install the neo4j-driver package
!pip install neo4j-driver

In [24]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [25]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="123")

In [26]:
conn.query("CREATE OR REPLACE DATABASE Tweets")

[]

### Create Constraints
The below constraints are created to ensure uniqueness of the data created.

In [27]:
create_constraint_query = '''
CREATE CONSTRAINT ON (t:Tweet) ASSERT t.tweetID IS UNIQUE;
CREATE CONSTRAINT ON (u:User) ASSERT u.userID IS UNIQUE;
CREATE CONSTRAINT ON (h:Hashtag) ASSERT h.tag IS UNIQUE;
CREATE CONSTRAINT ON (l:Link) ASSERT l.url IS UNIQUE;
CREATE CONSTRAINT ON (s:Source) ASSERT s.source IS UNIQUE
'''

In [ ]:
conn.query(create_constraint_query, db='Tweets')

### Create Tweet node
The Tweet node contains:  
**tweetID**: id of the tweet   
**text**: content of the tweet  
**action**: obtained through "verb" (post = original; share = retweet)   
**link**: the link to the tweet   
**postedTime**: the posted time of the tweet   
**datePublished**: it is extracted from postedTime and used later in Q2 of Part 3 query.

In [35]:
create_tweet_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.id AS tweetID,
datetime({ epochMillis: apoc.date.parse(value.postedTime, "ms",
"yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'")}) AS postedTime,
value.text AS text,
value.verb AS verb,
value.link AS link
MERGE (t:Tweet{tweetID:tweetID}) 
ON CREATE SET
t.postedTime = postedTime,
t.datePublished = date(postedTime),
t.text = text,
t.action = verb,
t.link = link',
{batchSize:500}) YIELD * ; 
'''

In [ ]:
conn.query(create_tweet_query, db='Tweets')

### Create Retweeted Tweets and **\[Tweet RETWEETS Tweet\]** relationship

Firstly, retweeted tweets are identified from the json file through the user's action stored in ("verb": "share"). Then, the information of the original retweeted tweets are extracted from **"object"** and stored in the Tweet Node.  
  
The Tweet node contains:  
**tweetID**: id of the retweeted tweet   
**text**: content of the retweeted tweet  
**action**: it is obtained from "verb" in json file and is "post"^          
**link**: the link to the retweeted tweet  
**postedTime**: the post time of the retweeted tweet   
**datePublished**: it is extracted from postedTime and used later in Q2 of Part 3 query.  
*^Note: From json file, for retweeted tweets, the action is "post".   
For instance, User1 shared Tweet1 which was a retweet of Tweet2 posted by User2. Therefore, the action of tweet posted by User1 is "share" and the action of tweet posted by User2 is "post".*  

In [37]:
create_retweet_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.id AS tweetID,
value.object.id AS retweetID,
datetime({epochMillis: apoc.date.parse(value.object.postedTime, "ms", 
"yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'")}) AS postedTime,
Date(datetime({epochMillis: apoc.date.parse(value.object.postedTime, "ms", 
"yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'")})) AS datePublished, 
value.object.text AS text,
value.object.verb AS verb,
value.object.link AS link
MATCH (t:Tweet{tweetID:tweetID})
WHERE t.action = "share"
MERGE (rt:Tweet{tweetID:retweetID, postedTime:postedTime, datePublished :datePublished , text:text, action:verb, link:link}) 
MERGE (rt)<-[:RETWEETS]-(t)',
{batchSize:500}) YIELD * ; 
'''

In [ ]:
conn.query(create_retweet_query, db='Tweets')

### Create Link node and **\[Tweet CONTAINS Link\]** relationship
The Link node contains:  
**url**: the link(s) embedded in the tweet, NOT the link to the tweet itself.   
The url is extracted from "expanded_url" in "twitter_entities".

In [39]:
create_link_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.twitter_entities AS e, 
value.id AS tweetID
MATCH(t:Tweet{tweetID:tweetID}) 
FOREACH (u IN e.urls |
MERGE (url:Link {url:u.expanded_url})
MERGE (url)<-[:CONTAINS]-(t))', 
{batchSize:500}) YIELD * ;
'''

In [ ]:
conn.query(create_link_query, db='Tweets')

### Create Link node and **\[Tweet CONTAINS Link\]** relationship for *Retweeted Tweets*
*Note: Retweeted tweets data is stored in "object", therefore, it is created separately to establish the relationship for retweeted tweets.*

In [ ]:
create_rt_link_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.object.twitter_entities AS e, 
value.object.id AS retweetID
MATCH(t:Tweet{tweetID:retweetID}) 
FOREACH (u IN e.urls |
MERGE (url:Link {url:u.expanded_url})
MERGE (url)<-[:CONTAINS]-(t))', 
{batchSize:500}) YIELD * ; 
'''

In [ ]:
conn.query(create_rt_link_query, db='Tweets')

### Create Source node and **\[Tweet USING Source\]** relationship
Source node contains:  
**source**: the app or site to post or share a tweet

In [ ]:
create_source_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 'WITH 
value.generator.displayName AS source, value.id AS tweetID
MATCH(t:Tweet{tweetID:tweetID}) 
MERGE (s:Source{source:source}) MERGE (t)-[:USING]->(s)', 
{batchSize:500}) YIELD * ; 
'''

In [ ]:
conn.query(create_source_query, db='Tweets')

### Create Source node and **\[Tweet USING Source\]** relationship for *Retweeted Tweets*
*Note: Retweeted tweets data is stored in "object", therefore, it is created separately to establish the relationship for retweeted tweets.*

In [ ]:
create_rt_source_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 'WITH 
value.object.generator.displayName AS source, value.object.id AS retweetID
MATCH(t:Tweet{tweetID:retweetID}) 
MERGE (s:Source{source:source}) MERGE (t)-[:USING]->(s)', 
{batchSize:500}) YIELD * ;
'''

In [ ]:
conn.query(create_rt_source_query, db='Tweets')

### Create User node and **\[User POSTS Tweet\]** relationship
User node contains:  
**userID**: id of the user  
**userName**: preferred user name of the user.   
"preferredUsername" is used in order to ensure it is as same as the name referred by others when mentioning him or her in a tweet.

In [ ]:
create_user_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.actor.id AS userID, 
value.actor.preferredUsername AS userName, value.id AS tweetID
MATCH(t:Tweet{tweetID:tweetID}) 
MERGE (u:User{userID:userID, userName:userName}) 
MERGE (u)-[:POSTS]->(t)', 
{batchSize:500}) YIELD * ;
'''

In [ ]:
conn.query(create_user_query, db='Tweets')

### Create User node and **\[User POSTS Tweet**\] relationship for *Retweeted Tweets*
Extract the users for the original retweeted tweets.  
*Note: Retweeted tweets data is stored in "object", therefore, it is created separately to establish the relationship for retweeted tweets.*

In [ ]:
create_rt_user_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.object AS o,
value.object.id AS retweetID
MATCH(t:Tweet{tweetID:retweetID})
FOREACH(a IN o.actor | 
MERGE (u:User{userID:a.id, userName:a.preferredUsername})
MERGE (u)-[:POSTS]->(t))', 
{batchSize:500}) YIELD * ; 
'''

In [ ]:
conn.query(create_rt_user_query, db='Tweets')

### Create Hashtag node and **\[Tweet TAGS Hashtag**\] relationship
Hashtag Node contains:  
**tag**: tag(s) used in the tweet

In [ ]:
create_hashtag_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.twitter_entities AS e, 
value.id AS tweetID
MATCH(t:Tweet{tweetID:tweetID}) 
FOREACH (h IN e.hashtags |
MERGE (tag:Hashtag {tag:TOLOWER(h.text)})
MERGE (tag)<-[:TAGS]-(t))', 
{batchSize:500}) YIELD * ; 
'''

In [ ]:
conn.query(create_hashtag_query, db='Tweets')

### Create Hashtag node and **\[Tweet TAGS Hashtag**\] relationship for *Retweeted Tweets*
*Note: Retweeted tweets data is stored in "object", therefore, it is created separately to establish the relationship for retweeted tweets.*

In [ ]:
create_rt_hashtag_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.object.twitter_entities AS e,
value.object.id AS retweetID
MATCH(t:Tweet{tweetID:retweetID})
FOREACH (h IN e.hashtags |
MERGE (tag:Hashtag {tag:TOLOWER(h.text)})
MERGE (tag)<-[:TAGS]-(t))', 
{batchSize:500}) YIELD * ; 
'''

In [ ]:
conn.query(create_rt_hashtag_query, db='Tweets')

## The below is addressing Q4 of Part 3 Query. 
It is presented here as to create the complete graph data model as shown at the beginning of Part 2.

### Create **\[Tweet MENTIONS User**\] relationship

In [ ]:
create_mentions_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.twitter_entities AS e, 
value.id AS tweetID
MATCH(t:Tweet{tweetID:tweetID}) 
FOREACH (m IN e.user_mentions |
MERGE (mentioned:User {userID:m.id, userName:m.screen_name})
MERGE (mentioned)<-[:MENTIONS]-(t))', 
{batchSize:500}) YIELD * ;
'''

In [ ]:
conn.query(create_mentions_query, db='Tweets')

### Create **\[Tweet MENTIONS User**\] relationship for *Retweeted Tweets*
*Note: Retweeted tweets data is stored in "object", therefore, it is created separately to establish the relationship for retweeted tweets.*

In [ ]:
create_rt_mentions_query = '''
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.object.twitter_entities AS e, 
value.object.id AS retweetID
MATCH(t:Tweet{tweetID:retweetID}) 
FOREACH (m IN e.user_mentions |
MERGE (mentioned:User {userID:m.id, userName:m.screen_name})
MERGE (mentioned)<-[:MENTIONS]-(t))', 
{batchSize:500}) YIELD * ; 
'''

In [ ]:
conn.query(create_rt_mentions_query, db='Tweets')

### Create **\[User FOLLOWS User**\] relationship
Since we have already included all retweeted tweet in the Tweet node and created all relationship for retweeted tweet, we are able to create FOLLOWS relationship in one go. Assigned weight = 1 to every FOLLOWS relationship. The weight is to be used in addressing Q5 of Part 3 Query.  

In [ ]:
create_follows_query = '''
MATCH (u1:User)-[:POSTS]->(t:Tweet)-[:MENTIONS]->(u2:User)
MERGE (u1)-[:FOLLOWS {weight: 1}]->(u2)
'''

In [ ]:
conn.query(create_follows_query, db='Tweets')

# Task 3: Query
Write Cypher statements in Neo4j.

#### 1. Find the top five most used sources (app or site) to post or share a tweet. For each source return the source name and the number of tweets sent via that source.

In [ ]:
MATCH (t:Tweet)-[:USING]->(s:Source)
WITH collect(t) AS tweets, s
RETURN s.source AS `Source Name`, size(tweets) AS `Number of Tweets` ORDER BY size(tweets) DESC LIMIT 5

Since Tweet node contains both tweets and retweeted tweets, the above cypher statement will return the top five most popular sources used in all tweets and retweeted tweets. For instance, User1 use Facebook to share tweet1 which is a retweet of tweet2 posted by User2 using TweetDeck.

| Source Name         | Number of Tweets |
|---------------------|------------------|
| Twitter for iPhone  | 4343             |
| Twitter Web Client  | 2805             |
| Twitter for Android | 1761             |
| TweetDeck           | 913              |
| Twitter for iPad    | 467              |

#### 2. Find the top five most used hashtags across all tweets on each day between 26th and 31st March 2016 (inclusive). For each day, return the date and a list of the five top hashtags in order of popularity on that day.

In [ ]:
MATCH (h:Hashtag)<-[r1:TAGS]-(t:Tweet)-[:RETWEETS]->(rt:Tweet)
WITH t, rt.datePublished AS Date, collect(t) AS tweets, collect(h.tag) AS tags 
WHERE Date > date("2016-03-25") AND Date < date("2016-04-01") 
UNWIND tags AS tag 
WITH Date AS Date, tag AS tag, size(tweets) AS num
ORDER BY num DESC
RETURN Date AS Date, collect(tag)[..5] AS Tags ORDER BY Date 

| Date       | Tags                                                         |
|:-----------|:-------------------------------------------------------------|
| 2016-03-26 | ["brands", "socialmedia", "timemanagement", "auspol", "nswpol"]          |
| 2016-03-27 | ["isis", "brusselsattacks", "brussels", "islam", "pjnet"]                |
| 2016-03-28 | ["fighthunger", "walmart", "ctustrike", "lfc", "vibewithus"]             |
| 2016-03-29 | ["model", "suicidegirls", "suicide_girls", "tattoo", "tattoos"]          |
| 2016-03-30 | ["australia", "ntaustralia", "darwin", "tourismtopend", "territorylife"] |
| 2016-03-31 | ["aprilfools", "happyfoolday", "world", "foolsday", "april"]             |

#### 3. Find all users that use any of the same hashtags as user "m_mrezamm". This query must exclude any retweets since these posts would automatically contain common tags. The query must return the user name and the number of hashtags that were used in their tweets that are also used by "m_mrezamm". Order results by the number of hashtags used in common. 

In [ ]:
MATCH (m:User)-[:POSTS]->(mt:Tweet)-[:TAGS]->(mh:Hashtag)
WHERE m.userName = 'm_mrezamm'
WITH collect(DISTINCT mh.tag) as commonTags
MATCH (u:User)-[:POSTS]->(t:Tweet)-[:TAGS]->(h:Hashtag)
WHERE t.action = 'post' AND h.tag in commonTags AND NOT u.userName = 'm_mrezamm'
UNWIND commonTags as ctags
WITH ctags, h, u, t
WHERE h.tag = ctags
RETURN DISTINCT u.userName AS `User Name`, collect(h.tag) AS Tags, count(h.tag) AS `Count` ORDER BY `Count` DESC

To exclude any retweets, the action:"post" is used to filter away any shared tweets, so that only original tweets are included in the result. The above cypher statement also return the tags used by the users to see which tags they used were the same as the tags used by m_mrezamm.

| User Name       | Tags                                                | Count |
|:----------------|:----------------------------------------------------|-------|
| paydaran        | ["iran", "humanrights", "unhcr", "no2rouhani", "eupol", "canada"] | 6     |
| boghche         | ["iran", "iran", "paris"]                                   | 3     |
| parnian2013     | ["iran", "rohani", "autriche"]                              | 3     |
| bhardost        | ["iran", "iran"]                                          | 2     |
| Mojahedineng    | ["iran"]                                                | 1     |
| CSDHI           | ["iran"]                                                | 1     |
| Darvish_Roghani | ["iran"]                                                | 1     |
| YahooBHRP       | ["humanrights"]                                         | 1     |
| hropenebook     | ["humanrights"]                                         | 1     |
| syndicalisms    | ["paris"]                                               | 1     |
| RavenHUWolf     | ["paris"]                                               | 1     |
| behroozsalam   | ["paris"]                                               | 1     |

#### 4. The original dataset does not contain information about which users follow each other. For this exercise we will infer that any user that MENTIONS another user in a tweet FOLLOWS that user. Write a Cypher expression which creates FOLLOW relationships based on this assumption, for example if UserA mentions UserB in a tweet, then it is assumed that UserA FOLLOWS UserB. Each FOLLOWS relationship added to the graph should also have a ‘weight’ property with a value of 1.

It was created in part 2. The below are recap of the codes:


[From Part 2] 12. Create **\[Tweet MENTIONS User\]** relationship

In [ ]:
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.twitter_entities AS e, 
value.id AS tweetID
MATCH(t:Tweet{tweetID:tweetID}) 
FOREACH (m IN e.user_mentions |
MERGE (mentioned:User {userID:m.id, screen_name:m.screen_name})
MERGE (mentioned)<-[:MENTIONS]-(t))', 
{batchSize:500}) YIELD * ; 

[From Part 2] 13. Create **\[Tweet MENTIONS User\]** relationship for Retweeted Tweets

In [ ]:
CALL apoc.periodic.iterate(
'CALL apoc.load.json("file:///cleaned_10000_tweets.json") YIELD value', 
'WITH value.object.twitter_entities AS e, 
value.object.id AS retweetID
MATCH(t:Tweet{tweetID:retweetID}) 
FOREACH (m IN e.user_mentions |
MERGE (mentioned:User {userID:m.id, screen_name:m.screen_name})
MERGE (mentioned)<-[:MENTIONS]-(t))', 
{batchSize:500}) YIELD * ; 

[From Part 2] 14. Create **\[User FOLLOWS User\]** relationship

In [ ]:
MATCH (u1:User)-[:POSTS]->(t:Tweet)-[:MENTIONS]->(u2:User)
MERGE (u1)-[:FOLLOWS {weight: 1}]->(u2)

#### 5. Using the FOLLOWS relationship derived in Problem 4, use the Neo4j Graph Data Science library to calculate the most popular nodes using Degree Centrality from the FOLLOWS subgraph. Then, find the top five users with the highest Degree Centrality score. (consider using NATURAL orientation and the weight property for the graph projection). 

In [ ]:
CALL gds.graph.project(
 'myGraph',
 'User',
 {
 FOLLOWS: {
 orientation: 'NATURAL',
 properties: ['weight']
 }
 }
)

In [ ]:
CALL gds.degree.stream('myGraph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).userName AS Name, score AS Followers
ORDER BY Followers DESC, Name DESC LIMIT 5

| Name            | Followers |
|-----------------|-----------|
| BlakeGeoff      | 35.0      |
| ejlazar         | 34.0      |
| Twittblaster    | 33.0      |
| SamanthaAppley4 | 27.0      |
| AlexiaVass      | 27.0      |

# YouTube Link

https://youtu.be/l6o3g9QMA-o